In [3]:
!pip install opencv-python

In [5]:
import os
import numpy as np
import cv2
from os.path import join
from glob import glob
import matplotlib.pyplot as plt
import skimage
import time
from math import sqrt
from tqdm.notebook import tqdm
from skimage.morphology import skeletonize
from os.path import basename
from concurrent.futures import ThreadPoolExecutor 

In [6]:
train_dir = './fingerprint/train.zip'
test_dir = './fingerprint/test2.zip'

In [7]:
import zipfile
with zipfile.ZipFile(train_dir, 'r') as zip_ref:
    zip_ref.extractall('./fingerprint')

In [8]:
import zipfile
with zipfile.ZipFile(test_dir, 'r') as zip_ref:
    zip_ref.extractall('./fingerprint')

In [9]:
train_dir = './fingerprint/train_ref'
test_dir = './fingerprint/test2'

In [10]:
train_images = [os.path.join(train_dir, f) for f in os.listdir(train_dir) if f.endswith('.BMP')]
test_images = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if f.endswith('.BMP')]

In [11]:
#이미지의 각 픽셀 값에서 평균을 빼고, 표준 편차로 나누어 정규화하는 함수 
def normalise(img):
    return (img-np.mean(img))/(np.std(img))

In [12]:
def segmentation(im, w=4, threshold=.2):
    (y, x) = im.shape
    threshold = np.std(im)*threshold

    image_variance = np.zeros(im.shape)
    segmented_image = im.copy()
    mask = np.ones_like(im)

    #w*w 블록 단위로 표준 편차 계산 
    for i in range(0, x, w):
        for j in range(0, y, w):
            box = [i, j, min(i + w, x), min(j + w, y)]
            block_stddev = np.std(im[box[1]:box[3], box[0]:box[2]])
            image_variance[box[1]:box[3], box[0]:box[2]] = block_stddev

    # 임계값 적용 
    mask[image_variance < threshold] = 0

    # smooth mask with a open/close morphological filter
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(w*2, w*2))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # 이미지 정규화 
    segmented_image *= mask

    return segmented_image

In [13]:
def binarization(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #gray scale로 변환 
  img = cv2.resize(img, (256, 256))  # 크기 조정
  img = cv2.GaussianBlur(img, (5, 5), 0)


  binary_image = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)
  return binary_image


In [14]:
def thinning_morph(image, kernel):
    #image = remove_small_objects(image, min_size=50)

    thining_image = np.zeros_like(image)
    img = image.copy()

    while 1:
        erosion = cv2.erode(img, kernel, iterations = 1) #침식, 객체 축소
        dilatate = cv2.dilate(erosion, kernel, iterations = 1) #팽창, 객체 확장 

        subs_img = np.subtract(img, dilatate) #팽창된 이미지와 차이 계산
        cv2.bitwise_or(thining_image, subs_img, thining_image) #subs_img를 thining_image (0 행렬)에 병합 
        img = erosion.copy() #img = 침식 이미지로 update

        done = (np.sum(img) == 0) #img 합이 0이 되면 종료 

        if done:
          break

    # thining_image를 한 픽셀 아래로 이동 
    down = np.zeros_like(thining_image)
    down[1:-1, :] = thining_image[0:-2, ]
    down_mask = np.subtract(down, thining_image)
    down_mask[0:-2, :] = down_mask[1:-1, ]
    #cv.imshow('down', down_mask)

    # thining_image를 한 픽셀 오른쪽으로 이동 
    left = np.zeros_like(thining_image)
    left[:, 1:-1] = thining_image[:, 0:-2]
    left_mask = np.subtract(left, thining_image)
    left_mask[:, 0:-2] = left_mask[:, 1:-1]
    #cv.imshow('left', left_mask)

    # 각 마스크 생성 후 결합하여 thining image 생성 
    cv2.bitwise_or(down_mask, down_mask, thining_image)
    output = np.zeros_like(thining_image)
    output[thining_image < 250] = 255

    return output

In [15]:
def preprocess_image(image_path):
  image = cv2.imread(image_path) 
  binary_image = binarization(image)
  # 이진화
  segmented_image = segmentation(binary_image)
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
  
  output = thinning_morph(segmented_image,kernel)
  
  
  return  output


In [16]:
def minutiae_at(pixels, i, j, kernel_size):
    #crossing number 방법을 사용해서 end와 bif를 찾는다. 

    # 가운데 픽셀이 검정색이라면 
    if pixels[i][j] == 1:
        #주변 픽셀 위치 
        cells = [(-1, -1), (-1, 0), (-1, 1),        # p1 p2 p3
                (0, 1),  (1, 1),  (1, 0),            # p8    p4
                (1, -1), (0, -1), (-1, -1)]           # p7 p6 p5
        #주변 픽셀 값 저장 
        values = [pixels[i + l][j + k] for k, l in cells]

        # 0에서 1로 또는 1에서 0으로의 변화를 계산 
        crossings = 0
        for k in range(0, len(values)-1):
            crossings += abs(values[k] - values[k + 1])
        crossings //= 2

        # 교차 수가 1이면 end point
        # 교차 수가 3이면 bifurcation point
        if crossings == 1:
            return "ending"
        if crossings == 3:
            return "bifurcation"

    return "none"


def calculate_minutiaes(img, kernel_size=3):
    biniry_image = np.zeros_like(img)
    biniry_image[img<10] = 1.0
    biniry_image = biniry_image.astype(np.int8)

    (y, x) = img.shape
    result = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    colors = {"ending" : (255, 0, 0), "bifurcation" : (0, 0, 255)}

    # iterate each pixel minutia
    for i in range(kernel_size//2, x - kernel_size//2):
        for j in range(kernel_size//2, y - kernel_size//2):
            # ROI 바깥 영역인 경우 무시
            #if mask[j,i]==0 
                #continue
            minutiae = minutiae_at(biniry_image, j, i, kernel_size)
            if minutiae != "none":
                cv2.circle(result, (i,j), radius=2, color=colors[minutiae], thickness=-2)

    return result

In [17]:
img = preprocess_image(test_images[1])

In [18]:
calculate_minutiaes(img)

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [19]:
def extract_minutiae_points(img, kernel_size=3):
    biniry_image = np.zeros_like(img)
    biniry_image[img < 10] = 1.0
    biniry_image = biniry_image.astype(np.int8)

    (y, x) = img.shape
    minutiae_points = {"ending": [], "bifurcation": []}

    for i in range(kernel_size//2, x - kernel_size//2):
        for j in range(kernel_size//2, y - kernel_size//2):
            minutiae = minutiae_at(biniry_image, j, i, kernel_size)
            if minutiae != "none":
                minutiae_points[minutiae].append((i, j))

    return minutiae_points

In [ ]:
p_test_images = []
for image_path in tqdm(test_images):
  preprocessed_image = preprocess_image(image_path)
  p_test_images.append(preprocessed_image)

In [ ]:
p_train_images = []
for image_path in tqdm(train_images):
  preprocessed_image = preprocess_image(image_path)
  p_train_images.append(preprocessed_image)

In [ ]:
test_minutiaes = []
for test_image in tqdm(p_test_images):
  test_minutiae = extract_minutiae_points(test_image)
  test_minutiaes.append(test_minutiae)

In [ ]:
train_minutiaes = []
for train_image in tqdm(p_train_images):
  train_minutiae = extract_minutiae_points(train_image)
  train_minutiaes.append(train_minutiae)

In [ ]:
# 좌표 값들의 l2 norm distance 구하기 
def l2_norm_distance(points1, points2):
    if not points1 or not points2:
        return float('inf')
    points1 = np.array(points1)
    points2 = np.array(points2)
    dists = np.linalg.norm(points1[:, np.newaxis] - points2, axis=2)
    return np.mean(np.min(dists, axis=1))

def precompute_distances(test_minutiae, train_minutiaes):
    distances = []
    for idx, train_minutiae in enumerate(train_minutiaes):
        distance = 0
        if test_minutiae["ending"] and train_minutiae["ending"]:
            distance += l2_norm_distance(test_minutiae["ending"], train_minutiae["ending"])
        if test_minutiae["bifurcation"] and train_minutiae["bifurcation"]:
            distance += l2_norm_distance(test_minutiae["bifurcation"], train_minutiae["bifurcation"])
        
        print(f'{idx}: distance {min_dist}')

        distances.append(distance)
    return distances

def match_fingerprints(test_minutiae, train_minutiaes, kernel_size=3):
    precomputed_distances = precompute_distances(test_minutiae, train_minutiaes)
    best_match = np.argmin(precomputed_distances)
    min_distance = precomputed_distances[best_match]

    return best_match, min_distance


In [ ]:
import matplotlib.pyplot as plt

# 거리 분포 히스토그램 
plt.hist(distances, bins=20, color='skyblue', edgecolor='black')

plt.xlabel('Distance')
plt.ylabel('Frequency')
plt.title('Distribution of distances')

plt.show()


In [ ]:
for idx, test_minutiae in tqdm(enumerate(test_minutiaes)):
    best_match_idx, min_dist, distances = match_fingerprints(test_minutiae, train_minutiaes)
    test_image_name = test_images[idx].split("/")[-1].split(".")[0]
    train_image_name = train_images[best_match_idx].split("/")[-1].split(".")[0]
    print(f'Best match: Test image: {test_image_name} Train image {train_image_name} with distance {min_dist}')


In [ ]:
def match_fingerprints(test_image, train_images):
    best_match = None
    best_score = float('inf')
    test_image = np.array(test_image)
    for idx, train_image in enumerate(train_images):
        train_image = np.array(train_image)
        #if test_image.shape == train_image.shape:
        score = np.sum((test_image.astype("float") - train_image.astype("float")) ** 2)
        if score < best_score:
            best_score = score
            best_match = idx
    return best_match, best_score


In [ ]:
matches = {}
distances = {}

for test_name in test_images:
    best_match, best_distance = match_fingerprints(p_test_images, p_train_images)
    test_name = test_name.split("/")[-1].split(".")[0]
    train_name = train_images[best_match].split("/")[-1].split(".")[0]
   
    print(f"Best match for { test_name}: {train_name} with distance {best_score}")


In [ ]:
def calculate_metrics(distances, threshold):
    true_positives = sum(1 for distance in distances if distance < threshold)
    false_positives = sum(1 for distance in distances if distance >= threshold)
    false_negatives = len(distances) - true_positives
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    accuracy = true_positives / len(distances)
    
    FAR = false_positives / len(distances)
    FRR = false_negatives / len(distances)
    
    return precision, recall, FAR, FRR, accuracy

In [ ]:
threshold = 15
precision, recall, FAR, FRR, accuracy = calculate_metrics(distances, threshold)
print(f"Precision: {precision}, Recall: {recall}, FAR: {FAR}, FRR: {FRR}, Accuracy: {accuracy}")

In [ ]:
def calculate_metrics(test_minutiae, train_minutiaes, threshold):
    best_match, min_distance, precomputed_distances = match_fingerprints(test_minutiae, train_minutiaes)
    
    TP = sum(1 for dist in precomputed_distances if dist <= threshold)
    FP = sum(1 for dist in precomputed_distances if dist <= threshold) - 1 # Excluding the best match
    TN = sum(1 for dist in precomputed_distances if dist > threshold)
    FN = 0 if min_distance <= threshold else 1
    
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    recall = TP / (TP + FN) if TP + FN != 0 else 0
    FAR = FP / (FP + TN) if FP + TN != 0 else 0
    FRR = FN / (FN + TP) if FN + TP != 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    return precision, recall, FAR, FRR, accuracy